In [1]:
#test the ollama local setup

from llama_index.llms.ollama import Ollama

#designate llm
llm = Ollama(model="phi3", request_timeout=60.0)

#test llm response
response = llm.complete("What color is Guinness beer?")
print(response)

c:\Users\mharr\miniconda3\envs\mpepenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Guinness beer, specifically the dark stout variant, is famously known for its rich, dark brown color. This distinctive hue comes from the roasted barley used in its brewing process and gives it a unique appearance that distinguishes it from other types of beers.


In [14]:
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings

# local_model_path = r"C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Models\Phi-3-mini-4k-instruct-q4.gguf"  # Replace with the your local path

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name=local_model_path,
#     trust_remote_code=True
# )

In [2]:
#test olamma embedding

from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings

ollama_embedding = OllamaEmbedding(
    model_name="phi3",
    ollama_additional_kwargs={"mirostat": 0},
)

#uncomment to test
# pass_embedding = ollama_embedding.get_text_embedding_batch(
#     ["This is a passage!", "This is another passage"], show_progress=True
# )
# print(pass_embedding)

# query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
# print(query_embedding)

In [15]:
import json

def to_langchain_format(chapter):
    """
    Transforms a chapter dictionary into a standardized format expected by LangChain.

    Args:
        chapter (dict): A dictionary containing the keys 'chapter_title' and 'content'
                        which hold the title and content of the chapter, respectively.

    Returns:
        dict: A dictionary formatted for LangChain, with the title and cleaned content.
    """
    # Assuming 'content' is already cleaned
    return {
        "title": chapter["chapter_title"],
        "content": chapter["content"]  # already cleaned
    }



In [77]:
#data transformation for langchain format

import json

# # Load the cleaned data
# file_path = r'C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Notebooks\mpep_data_clean.json'
# with open(file_path, 'r', encoding='utf-8') as file:
#     cleaned_chapters = json.load(file)

# # Apply the transformation to each chapter
# processed_documents = [to_langchain_format(chapter) for chapter in cleaned_chapters]

# # Optionally, save the processed documents to a new file
# output_path = r'C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Notebooks\langchain_ready_data.json'
# with open(output_path, 'w', encoding='utf-8') as file:
#     json.dump(processed_documents, file, indent=4, ensure_ascii=False)

# print("Processed data ready for LangChain saved to:", output_path)


In [3]:
#langchain documents object

from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir=r"C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\Notebooks\langchainjson")
documents = reader.load_data()

In [8]:
# embeddings creation

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load documents and build index
index = VectorStoreIndex.from_documents(
    documents, 
    show_progress=True, 
    embed_model=OllamaEmbedding(model_name='phi3'), 
)

Generating embeddings: 100%|██████████| 829/829 [36:10<00:00,  2.62s/it]


In [9]:
# import
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [11]:
# save to disk

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("MPEPcollection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=ollama_embedding
)

In [ ]:
# load from disk
db2 = chromadb.PersistentClient(path=r"C:\Users\mharr\OneDrive\Documents\GitHub\MPEP_finetune\chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=ollama_embedding,
)

In [21]:
chat_engine = index.as_chat_engine(llm = Ollama(model="phi3", request_timeout=120.0))
response = chat_engine.chat("What are the utility requirements of a patent? Please explain in detail.")
print(response)

Utility requirements for a patent typically include novelty, non-obviousness, and usefulness. Specifically, an invention must have practical use or application to be considered useful, which is often referred to as utility. This means the invention should provide some identifiable benefit and be capable of being made or used in some kind of industry. The specifics can vary depending on jurisdiction, but these are the basic criteria for utility in patent law.


In [14]:
query_engine = index.as_query_engine()
response = query_engine.query("What are the utility requirements of a patent? Please explain in detail.")
display(Markdown(f"<b>{response}</b>"))

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [21]:
import chainlit as cl
from llama_index.core import (
    Settings,
    StorageContext,
    VectorStoreIndex,
    SimpleDirectoryReader
)
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine
from llama_index.core.callbacks import CallbackManager
from llama_index.core.service_context import ServiceContext

2024-05-05 14:55:26 - Loaded .env file
